In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# This notebook is to analyze token routing across experts in switch_base_8 model.

In [ ]:
!pip install spacy==3.7.2

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
import torch
from torch import nn
import transformers
import numpy as np

In [ ]:
!pip install accelerate
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


#Model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from transformers import AutoTokenizer, SwitchTransformersForConditionalGeneration

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/switch-base-8")
model = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8")
model.to(device)

SwitchTransformersForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): SwitchTransformersStack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): SwitchTransformersBlock(
        (layer): ModuleList(
          (0): SwitchTransformersLayerSelfAttention(
            (SelfAttention): SwitchTransformersAttention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): SwitchTransformersLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): SwitchTransformersLayerFF(
            (mlp): SwitchTransformersDenseActDense(
              (wi): Linear(in_features=768, out_features=3072

#Dataset - SyntaxGym

In [ ]:
from datasets import load_dataset
dataset = load_dataset("cpllab/syntaxgym_sentences")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [ ]:
wv.most_similar('mountain', topn=40)

[('mountains', 0.837950587272644),
 ('mountainside', 0.7189289927482605),
 ('mountaintop', 0.6948275566101074),
 ('slopes', 0.6819800734519958),
 ('mountainous', 0.6697976589202881),
 ('mountain_peaks', 0.6446480751037598),
 ('crag', 0.6418702602386475),
 ('alpine', 0.6341957449913025),
 ('hills', 0.6330642700195312),
 ('fourteener', 0.632472038269043),
 ('Colorado_fourteeners', 0.6307529211044312),
 ('hill', 0.6287425756454468),
 ('Huayna_Picchu', 0.6262496709823608),
 ('craggy_mountain', 0.6188591718673706),
 ('Mount_Timpanogos', 0.6140114665031433),
 ('Mount_Tallac', 0.612027645111084),
 ('alpine_peaks', 0.6095947027206421),
 ('Easily_scaling', 0.6095635890960693),
 ('foothills', 0.6080213785171509),
 ('moutain', 0.6075683236122131),
 ('Crestone_Needle', 0.6072766780853271),
 ('Khumbu_Icefall', 0.605628490447998),
 ('Khumbu_valley', 0.605059027671814),
 ('Cairn_Gorm', 0.5998322367668152),
 ('massif', 0.5991336703300476),
 ('Kala_Pattar', 0.5982056260108948),
 ('Vallee_Blanche', 0.59

In [ ]:
homonym_sentences = {
  "bank": "I went to the bank to withdraw money before heading to the river bank to enjoy the sunset.",
  "bat": "He bought a new bat for baseball, not realizing a bat was sleeping in the corner of the garage.",
  "bark": "The tree's bark was rough against my back as I listened to the dog bark excitedly in the distance.",
  "current": "While studying about electric current, she felt the strong current pull her as she swam.",
  "date": "I checked the calendar to see the date, wondering if my blind date would enjoy dates as a snack.",
  "die": "They used a single die to decide who would go first, not knowing fate would soon cast a darker die.",
  "fine": "After arguing that the weather was fine, he ended up paying a fine for speeding under the clear skies.",
  "lead": "They decided to lead the group, unaware of the health risks posed by the lead piping in the building.",
  "left": "After he left the room, I realized the only piece of cake left was his.",
  "match": "She lit the candle with a match, hoping its glow would match the mood she wanted to set.",
  "mean": "His intentions were mean, but the average (mean) of the numbers was surprisingly high.",
  "mold": "They found mold on the bread in the shape of a mold they had never seen.",
  "mussel": "She pulled a mussel from the net, feeling her muscle strain under the weight.",
  "pitch": "He decided to pitch his idea at a higher pitch to grab everyone's attention.",
  "plain": "She preferred plain yogurt, looking out over the plain, wishing for simpler times.",
  "right": "He turned right into the art gallery, convinced he was in the right place for inspiration.",
  "ring": "He heard a ring at the door, just as he finished polishing the ancient ring.",
  "row": "They had a row about how to row the boat effectively in the race.",
  "sewer": "The tailor complained about the sewer system, fearing it might affect his work in the sewer.",
  "tear": "A tear ran down her cheek as she attempted to tear the heavy paper.",
  "bow": "He took a bow after the performance, holding a bow and arrow in his hand for the next act.",
  "close": "They had to close the shop early to stay close to their family during the holiday.",
  "desert": "He promised he would never desert her, even in the vast desert where they were stranded.",
  "dove": "The dove soared above the water before she dove into the lake.",
  "fair": "He argued it was only fair to pay the entrance fee to the county fair.",
  "gill": "The fisherman checked the gill of the fish before sharing his favorite Gillian Flynn novel.",
  "jam": "She found herself in a traffic jam, realizing she had forgotten her favorite jam at the grocery store.",
  "lie": "It was no lie that she loved to lie in the sun.",
  "minute": "He asked for a minute of her time, not noticing the minute details of her work.",
  "number": "As the cold numbed her fingers, she realized her number was no longer listed.",
  "pound": "He decided to adopt a dog from the pound, feeling the weight of the decision pound in his heart.",
  "ruler": "The ruler of the land had a favorite ruler to measure the progress of his realm.",
  "scale": "The musician played a scale on her guitar, unaware of the fish scale stuck to her shoe.",
  "sole": "The sole of his shoe was worn out, making him the sole participant to finish the marathon barefoot.",
  "stair": "He raced up the stair, staring into the distance.",
  "steal": "He would never steal, even if it was to secure the steel needed for the project.",
  "tail": "The comet's tail was visible in the night sky, just as the cat's tail swept the table clear.",
  "tire": "She didn't tire of the view, even as she changed the tire on her car.",
  "waste": "It was a waste of time to argue, especially in the waste lands of the far north.",
  "wave": "He gave a wave to his friend, not noticing the giant wave approaching from behind.",
  "well": "She wished well for her friends, drawing water from the ancient well.",
  "wind": "The wind was too strong for sailing, making it difficult to wind the clock.",
  "wound": "The wound was deep, inflicted by the sharp wind that wound through the alley.",
  "bear": "He couldn't bear the weight, watching the bear roam through the campsite.",
  "crane": "She had to crane her neck to see the crane lifting materials at the construction site.",
  "draft": "He felt a cold draft in the room while drafting his novel.",
  "engaged": "They were engaged in conversation when he mentioned they had finally got engaged.",
  "lean": "He had to lean against the wall to support his lean figure.",
  "saw": "He saw the art piece in the gallery before deciding to saw the wood for his next project."
}


In [ ]:
import string
interchange_dict = {
    "dog": "A loyal dog waits patiently at the door for its owner to return.",
    "car": "The red car gleamed under the afternoon sun, freshly washed and polished.",
    "book": "She found an old book tucked away in the corner of the library.",
    "tree": "A solitary tree stood in the middle of the field, its branches reaching out.",
    "coffee": "His morning routine always included a strong cup of coffee to start the day.",
    "guitar": "The musician tuned his guitar carefully before the concert began.",
    "garden": "Their backyard garden was full of blooming flowers and buzzing bees.",
    "painting": "The museum featured a painting from the 18th century, still vibrant in color.",
    "river": "A gentle river flowed through the town, its waters clear and cool.",
    "bicycle": "She rode her bicycle through the park, enjoying the breeze and sunshine.",
    "hat": "He wore a wide-brimmed hat to protect himself from the harsh sun.",
    "mountain": "They planned to climb the mountain early in the morning to catch the sunrise.",
    "shoe": "I lost a shoe while running through the muddy trails yesterday.",
    "phone": "Her phone rang just as she was about to step into the shower.",
    "pen": "He always carried a pen and notebook for whenever inspiration struck.",
    "cake": "For her birthday, they baked a cake with layers of chocolate and vanilla.",
    "train": "The train whistled as it approached the small station at dawn.",
    "key": "She searched her bag for the key to the old cabin.",
    "light": "A single light flickered in the distance, guiding ships to the harbor.",
    "bird": "Early in the morning, a bird sang outside the window, a melody of nature.",
    "clock": "The clock struck midnight, echoing through the silent house.",
    "bridge": "The old bridge connected the two parts of the city, a link between past and present.",
    "camera": "He captured the moment with his camera, freezing time in a photograph.",
    "apple": "An apple fell from the tree, ripe and ready to be eaten.",
    "mirror": "In the mirror, she practiced her speech, determined to make it perfect.",
    "boat": "They decided to rent a boat and spend the day on the lake.",
    "pillow": "He muffled the alarm with a pillow, not ready to start the day.",
    "computer": "Her computer crashed right before she could save her work.",
    "rose": "He gave her a rose, a symbol of love and appreciation.",
    "map": "The old map led them to a hidden treasure buried beneath the oak.",
    "glass": "She accidentally dropped the glass, shattering it into countless pieces.",
    "letter": "A handwritten letter arrived in the mail, sealed with a wax stamp.",
    "star": "At night, they lay on the grass, gazing up at the star-filled sky.",
    "butterfly": "A butterfly landed on her hand, its wings a kaleidoscope of colors.",
    "chocolate": "He had a craving for chocolate, so he headed to the nearest store.",
    "sneaker": "Her new sneaker was perfect for running, both comfortable and stylish.",
    "umbrella": "They forgot an umbrella and got caught in the sudden downpour.",
    "bottle": "A message was found inside an old bottle washed up on the beach.",
    "notebook": "Every page of the notebook was filled with ideas and sketches.",
    "sunset": "The sunset painted the sky in hues of orange and pink, a breathtaking view.",
    "cookie": "In the jar, only one cookie remained, tempting those who passed by."
}


# Function to check if the word contains punctuation
def contains_punctuation(word):
    return any(char in string.punctuation for char in word)

# Define a limit for how many closest words you want to use for substitutions
closest_words_limit = 100

# Function to replace a word in a sentence
def replace_word_in_sentence(sentence, old_word, new_word):
    # Ensure the replacement maintains the original word's capitalization
    if old_word.istitle():
        new_word = new_word.title()
    return sentence.replace(old_word, new_word)

# Store the new sentences
interchanged_sentences = {}

for key_word, sentence in interchange_dict.items():
    try:
        # Find the closest words to the key word
        closest = wv.most_similar(key_word, topn=closest_words_limit)
        new_versions = []
        for similar_word, _ in closest:
            # Filter out compound words, phrases, and words with punctuation
            if '_' not in similar_word and ' ' not in similar_word and not contains_punctuation(similar_word):
                new_sentence = replace_word_in_sentence(sentence, key_word, similar_word)
                new_versions.append(new_sentence)
        interchanged_sentences[key_word] = new_versions
    except KeyError:
        # If the key word is not in the model's vocabulary

        continue


In [ ]:
interchanged_sentences

{'dog': ['A loyal dogs waits patiently at the door for its owner to return.',
  'A loyal puppy waits patiently at the door for its owner to return.',
  'A loyal pooch waits patiently at the door for its owner to return.',
  'A loyal cat waits patiently at the door for its owner to return.',
  'A loyal Rottweiler waits patiently at the door for its owner to return.',
  'A loyal beagle waits patiently at the door for its owner to return.',
  'A loyal pup waits patiently at the door for its owner to return.',
  'A loyal Pomeranian waits patiently at the door for its owner to return.',
  'A loyal canines waits patiently at the door for its owner to return.',
  'A loyal chihuahua waits patiently at the door for its owner to return.',
  'A loyal pet waits patiently at the door for its owner to return.',
  'A loyal schnauzer waits patiently at the door for its owner to return.',
  'A loyal dachshund waits patiently at the door for its owner to return.',
  'A loyal Doberman waits patiently at 

In [ ]:
from random import shuffle


sentences_to_be_scrambled = [
    "Sunset over the mountains casts a golden glow across the valley, enchanting all who see it.",
    "Amidst the city's heart, a quiet park provides a serene escape from its perpetual hustle and bustle.",
    "Towering ancient trees, with leaves whispering ancient tales, stand as guardians of history.",
    "A gentle breeze infused with jasmine scent wafts through an open window, enriching the room's ambiance.",
    "Echoes of laughter traverse the hallway, serving as a lively reminder of joy's simplicity.",
    "At night, stars twinkle like jewels, weaving tales of ancient constellations across the sky.",
    "Relentlessly, waves meet the shore, a testament to nature's enduring power and rhythm.",
    "Freshly baked bread's aroma spreads, enticing passersby with its homely allure.",
    "Across the sky, clouds meander lazily, their forms shape-shifting with the wind's whims.",
    "Marking time's passage, the old clock tower stands, its chimes echoing through the air.",
    "Shelves brim with books, each a gateway to worlds unknown and adventures untold.",
    "In a sunbeam's embrace, a cat stretches with contentment, basking in its warmth.",
    "Joyful children's laughter permeates the air, brightening the dreariest of days.",
    "As the full moon ascends, it bathes the landscape in a mystical, silvery luminescence.",
    "With bold and vibrant strokes, artists reimagine the world on their canvases.",
    "Streets resonate with melodies, weaving a tapestry of stories from faces unknown.",
    "A garden in full bloom offers a spectacle of colors, a testament to life's vibrancy.",
    "Petrichor, the scent of rain on parched earth, heralds a forthcoming storm's arrival.",
    "Marketplace hustle becomes a symphony of life, vibrant with sights, sounds, and scents.",
    "Morning's quiet is disturbed only by the gentle steam from a freshly brewed cup of coffee.",
    "Silence envelops the library, a sacred haven for seekers of knowledge and solace.",
    "Night's serenade is crafted by crickets, their chorus a comforting cloak in the darkness.",
    "A winding path through the forest invites on a journey of discovery with each step.",
    "Snowflakes descend softly, each a fleeting masterpiece of nature's delicate artistry.",
    "Moonbeams playing through leaves cast a dance of light and shadow upon the earth.",
    "Within the hearth, a fire crackles, offering its warm embrace against winter's chill.",
    "Dusk transforms the city skyline into a masterpiece of shadows and light.",
    "Footsteps echo in an empty corridor, whispering tales of solitude and time's relentless march.",
    "Morning air vibrates with birdsong, a natural chorus heralding the day's awakening.",
    "Bridging rivers, a structure connects worlds, uniting people and stories across its span.",
    "Pine's fresh scent in the air serves as a brisk reminder of the wild's allure.",
    "Chalk drawings by a street artist spring to life, capturing the imagination of all who pass.",
    "Flickering candle flames cast a ballet of shadows, silent yet expressive against the walls.",
    "After a storm, rainbows serve as vibrant symbols of hope and renewal amidst adversity.",
    "Yellowed pages of an old book whisper echoes from times long past, inviting immersion.",
    "Nature's might is showcased as a waterfall cascades with force down a steep cliff.",
    "Night festivals illuminate the darkness, celebrating community with a blaze of lights and joy.",
    "A painter's palette, cluttered with colors, tells tales of emotional journeys and creative visions.",
    "Sunlight's gentle caress on your face stands as a tender reminder of life's simple joys.",
    "Ruins of ancient civilizations whisper stories of eras gone, etched in stone and memory.",
    "Bounty of the earth is celebrated at the farmer's market, a feast for all senses.",
    "Under a tree's cool shade, one finds refuge from the summer's heat.",
    "A stream's murmur accompanies a day spent in nature, soothing and constant.",
    "Air filled with spices tells tales of distant lands and culinary traditions passed through generations.",
    "With grace and precision, a dancer narrates stories through movement, captivating the audience.",
    "Lanterns' soft glow creates an enchanting atmosphere, transforming the night with their light.",
    "A snow-covered landscape lies in hushed serenity, offering a peaceful retreat from the world's clamor.",
    "Moments captured by an artist's brush come to life, each stroke telling its own story.",
    "Dawn's first songbird melody acts as a gentle alarm, waking the world with its tune.",
    "City life's rhythm, a complex blend of chaos and order, thrives, ever vibrant and dynamic."
]

# Create a pipeline to shuffle each sentence and generate 4 variations
def shuffle_sentence(sentence):
    sentence_groups = []
    words = sentence.split()
    shuffled_sentences = [sentence]  # Original sentence
    for _ in range(3):  # Generate 3 scrambled versions
      shuffle(words)
      shuffled_sentences.append(' '.join(words))
    sentence_groups.append(shuffled_sentences)
    return sentence_groups

scrambled_sentences = [shuffle_sentence(sentence) for sentence in sentences_to_be_scrambled]


In [ ]:
scrambled_sentences

In [ ]:
sentences = dataset['train'][0:400]['sentence']

In [ ]:
same_word_sentences = {
  0: ["She deposited her paycheck at the bank this morning.", "The children played with toys at the bank.", "The bank where she works is closed today.", "We stopped by the bank to make a deposit."],
  1: ["The baby giggled as she rolled the toy across the floor.", "He enjoyed playing with his new toy truck.", "He stepped on a toy and tripped.", "The toy store was having a big sale."],
  2: ["His ultimate goal is to win the race.", "Scoring a goal in overtime won them the championship.", "She set a goal to improve her grades.", "They scored three goals in the first half."],
  3: ["Tomorrow is my birthday.", "It is going to be cold today so wear a jacket.", "Today is Friday.", "The weather is supposed to be nice this weekend."],
  4: ["Her work schedule is very demanding.", "I have a lot of work to do this weekend.", "He is at work right now.", "She works two jobs to pay the bills."],
  5: ["We played catch with the ball.", "The quarterback threw a perfect spiral ball down the field.", "The ball bounced off the rim.", "He kicked the ball into the goal."],
  6: ["She got a burn from the hot slide.", "The children laughed as they went down the slide.", "The playground had a really tall slide.", "He went down the slide head first."],
  7: ["It was an amazing trip.", "They are going on a trip to the beach next month.", "She tripped on the stairs.", "The field trip to the museum was fun."],
  8: ["He had a bad day at work.", "This food tastes really bad.", "The weather was bad so we stayed inside.", "That was a bad mistake."],
  9: ["You did a great job!", "I feel great now that I'm done with that project.", "That was a great movie.", "She's a great friend."],
  10: ["I would like to eat there again soon.", "I really like eating at that restaurant.", "Do you like horror movies?", "Her nails look like those of a monster."],
  11: ["The quarterback threw to the receiver.", "The receiver jumped high in the air to make the catch.", "He's a talented receiver.", "The receiver caught the ball and scored."],
  12: ["They crowned him king after the war.", "The king ruled over the kingdom justly.", "We learned about King Arthur and the Knights of the Round Table.", "The king lived in a grand castle."],
  13: ["He was hired by the CIA.", "Agents from the CIA were on a secret mission.", "The CIA uncovered the plot.", "She worked for the CIA for 20 years."],
  14: ["The view from the mountaintop was marvelous.", "Her home decorated so marvelously for the holidays.", "You look marvelous in that dress.", "He did a marvelous job on the presentation."],
  15: ["You have the right to bear arms.", "We saw a bear while hiking through the woods.", "He has a tattoo of a bear on his arm.", "To bear witness to injustice is to tell a story about our world based upon firsthand observation of real-life circumstance."],
  16: ["The team celebrated their champion quarterback's leadership.", "As the champion, he gave a speech at the ceremony.", "She was the champion for 5 years in a row.", "You have always been a champion of the people."],
  17: ["I need to charge my phone.", "You can call me on my phone when you arrive.", "You will sing thr national anthem, don't phone it in!", "I will phone the bank now"],
  18: ["We smelled smoke and called the fire department.", "There was a lot of smoke coming from the chimney.", "The fire was sending up plumes of black smoke.", "To smoke is to slowly kill yourself."],
  19: ["Stop it or the curtains will catch fire!", "Nice catch! You made it look easy.", "We tried to catch butterflies in the field.", "I don't trust you, what's the catch?"]
}

same_word_sentence_indices = {0: 'bank',1: 'toy', 2: 'goal', 3: 'is', 4: 'work', 5: 'ball', 6: 'slide', 7: 'trip', 8: 'bad', 9: 'great', 10: 'like', 11: 'receiver', 12: 'king', 13: 'CIA', 14: 'marvelous', 15: 'bear', 16: 'champion', 17: 'phone', 18: 'smoke', 19: 'catch'}

In [ ]:
from collections import defaultdict
indices_in_sentences = defaultdict(list)
for i, word in same_word_sentence_indices.items():
  for sentence in same_word_sentences[i]:
    indices_in_sentences[i].append(sentence.lower().index(word.lower()))

indices_in_sentences

defaultdict(list,
            {0: [34, 37, 4, 18],
             1: [35, 32, 16, 4],
             2: [13, 10, 10, 18],
             3: [9, 3, 6, 12],
             4: [4, 16, 9, 4],
             5: [25, 39, 4, 14],
             6: [28, 43, 33, 17],
             7: [18, 20, 4, 10],
             8: [9, 24, 16, 11],
             9: [10, 7, 11, 8],
             10: [8, 9, 7, 15],
             11: [29, 4, 16, 4],
             12: [17, 4, 17, 4],
             13: [20, 16, 4, 19],
             14: [34, 22, 9, 9],
             15: [22, 9, 21, 3],
             16: [26, 7, 12, 23],
             17: [20, 22, 41, 7],
             18: [11, 19, 40, 3],
             19: [29, 5, 12, 30]})

In [ ]:
same_word_sentence_indices.values()

dict_values(['bank', 'toy', 'goal', 'is', 'work', 'ball', 'slide', 'trip', 'bad', 'great', 'like', 'receiver', 'king', 'CIA', 'marvelous', 'bear', 'champion', 'phone', 'smoke', 'catch'])

In [ ]:
close_word_sentences = {
  "1": ["The king ruled the kingdom wisely.", "The queen ruled the kingdom wisely.", "The emperor ruled the kingdom wisely.", "The empress ruled the kingdom wisely."],
  "2": ["The uncle took the kids to the zoo.", "The aunt took the kids to the zoo.", "The father took the kids to the zoo.", "The mother took the kids to the zoo."],
  "3": ["The gentleman held the door open for her.", "The lady held the door open for her.", "The man held the door open for her.", "The woman held the door open for her."],
  "4": ["The brother played video games all day.", "The sister played video games all day.", "The boy played video games all day.", "The girl played video games all day."],
  "5": ["The grandfather told stories about his childhood.", "The grandmother told stories about her childhood.", "The grandpa told stories about his childhood.", "The grandma told stories about her childhood."],
  "6": ["The nephew asked for money for his birthday.", "The niece asked for money for her birthday.", "The grandson asked for money for his birthday.", "The granddaughter asked for money for her birthday."],
  "7": ["The prince searched for a princess to marry.", "The princess searched for a prince to marry.", "The bachelor searched for a bachelorette to marry.", "The bachelorette searched for a bachelor to marry."],
  "8": ["The wizard cast a magic spell.", "The witch cast a magic spell.", "The sorcerer cast a magic spell.", "The sorceress cast a magic spell."],
  "9": ["The actor gave an incredible performance.", "The actress gave an incredible performance.", "The male actor gave an incredible performance.", "The female actress gave an incredible performance."],
  "10": ["The guy drove an expensive sports car.", "The gal drove an expensive sports car.", "The man drove an expensive sports car.", "The woman drove an expensive sports car."],
  "11": ["The groom looked handsome in his tuxedo.", "The bride looked beautiful in her wedding dress.", "The husband looked handsome in his tuxedo.", "The wife looked beautiful in her wedding dress."],
  "12": ["The prince lived in the castle.", "The princess lived in the castle.", "The prince lived in the palace.", "The princess lived in the palace."],
  "13": ["The father built a treehouse for the kids.", "The mother built a treehouse for the kids.", "The dad built a treehouse for the kids.", "The mom built a treehouse for the kids."],
  "14": ["The husband cleaned the dishes after dinner.", "The wife cleaned the dishes after dinner.", "The man cleaned the dishes after dinner.", "The woman cleaned the dishes after dinner."],
  "15": ["The boy studied hard for the test.", "The girl studied hard for the test.", "The son studied hard for the test.", "The daughter studied hard for the test."],
  "16": ["The male swimmer won first place.", "The female swimmer won first place.", "The man swimmer won first place.", "The woman swimmer won first place."],
  "17": ["The boyscout helped the old lady.", "The girlscout helped the old lady.", "The boyscout helped the elderly woman.", "The girlscout helped the elderly woman."],
  "18": ["The son mowed the lawn yesterday.", "The daughter mowed the lawn yesterday.", "The boy mowed the lawn yesterday.", "The girl mowed the lawn yesterday."],
  "19": ["The man ordered steak for dinner.", "The woman ordered steak for dinner.", "The gentleman ordered steak for dinner.", "The lady ordered steak for dinner."],
  "20": ["The uncle gave great gifts at Christmas.", "The aunt gave great gifts at Christmas.", "The grandfather gave great gifts at Christmas.", "The grandmother gave great gifts at Christmas."]
}

**New Hook**

In [ ]:
def router_forward_hook(layer_id, layer_type, token_routing):
    def hook(module, input, output):
        global original_input_ids, batch_number
        expert_index = output[1][1]

        batch_size, seq_length = expert_index.shape
        for batch_idx in range(batch_size):
            for token_idx in range(seq_length):
                token_id = original_input_ids[batch_idx, token_idx].item()
                token_key = (token_id, token_idx)  # Unique identifier for each token occurrence

                if batch_number not in token_routing:
                  token_routing[batch_number] = {}
                if batch_idx not in token_routing[batch_number]:
                    token_routing[batch_number][batch_idx] = {}
                if token_key not in token_routing[batch_number][batch_idx]:
                    token_routing[batch_number][batch_idx][token_key] = {"encoder": [], "decoder": []}

                route = (layer_id, expert_index[batch_idx, token_idx].item())
                token_routing[batch_number][batch_idx][token_key][layer_type].append(route)

    return hook


In [ ]:
hooks = []
# Attach the router forward hooks to the encoder layers
token_routing = {}
for i, block in enumerate(model.encoder.block):
      if i % 2 == 1:
        hook_handle = block.layer[1].mlp.register_forward_hook(router_forward_hook(i, "encoder", token_routing))
        hooks.append(hook_handle)

# for i, block in enumerate(model.encoder.block):
#       if i % 2 == 1:
#         hook = router_forward_hook(i, "encoder", input_ids, token_routing)
#         block.layer[1].mlp.router.register_forward_hook(hook)

In [ ]:
original_input_ids = None
batch_number = 0

# Function to process a batch of sentences
def process_batch(batch_sentences):
    # Tokenize the input sentences
    global original_input_ids
    inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True)

    # Run the model
    with torch.no_grad():
        original_input_ids = inputs['input_ids'].to(device)
        model.generate(original_input_ids, max_length=20)

# Process the sentences in batches of 4
# sentences = []
# for group in list(same_word_sentences.values()):
#   sentences += group

sentences = scrambled_sentences


batch_size = 4
for i in range(len(sentences)):
    batch_number = i // 4
    batch_sentences = sentences[i][0]
    process_batch(batch_sentences)

# # Print or analyze the routing decisions
# print(token_routing)

In [ ]:
scrambled_sentences[1]

[["Amidst the city's heart, a quiet park provides a serene escape from its perpetual hustle and bustle.",
  "serene city's a the quiet bustle. provides its from Amidst a park perpetual escape and hustle heart,",
  "the a provides bustle. serene a quiet heart, its escape perpetual Amidst park city's and hustle from",
  "city's the a bustle. serene from Amidst quiet escape perpetual heart, hustle and a provides its park"]]

In [ ]:
for hook in hooks:
    hook.remove()


In [ ]:
print(sentences)

In [ ]:
same_word_sentences.values()

dict_values([['She deposited her paycheck at the bank this morning.', 'The children played with toys at the bank.', 'The bank where she works is closed today.', 'We stopped by the bank to make a deposit.'], ['The baby giggled as she rolled the toy across the floor.', 'He enjoyed playing with his new toy truck.', 'He stepped on a toy and tripped.', 'The toy store was having a big sale.'], ['His ultimate goal is to win the race.', 'Scoring a goal in overtime won them the championship.', 'She set a goal to improve her grades.', 'They scored three goals in the first half.'], ['Tomorrow is my birthday.', 'It is going to be cold today so wear a jacket.', 'Today is Friday.', 'The weather is supposed to be nice this weekend.'], ['Her work schedule is very demanding.', 'I have a lot of work to do this weekend.', 'He is at work right now.', 'She works two jobs to pay the bills.'], ['We played catch with the ball.', 'The quarterback threw a perfect spiral ball down the field.', 'The ball bounced 

In [ ]:
token_routing

{0: {0: {(27,
    0): {'encoder': [(1, 7),
     (3, 1),
     (5, 2),
     (7, 6),
     (9, 5),
     (11, 5)], 'decoder': []},
   (877, 1): {'encoder': [(1, 4), (3, 4), (5, 1), (7, 6), (9, 1), (11, 6)],
    'decoder': []},
   (12, 2): {'encoder': [(1, 0), (3, 5), (5, 0), (7, 5), (9, 7), (11, 3)],
    'decoder': []},
   (8, 3): {'encoder': [(1, 3), (3, 1), (5, 2), (7, 3), (9, 7), (11, 5)],
    'decoder': []},
   (2137, 4): {'encoder': [(1, 4), (3, 3), (5, 3), (7, 4), (9, 3), (11, 1)],
    'decoder': []},
   (12, 5): {'encoder': [(1, 0), (3, 7), (5, 0), (7, 5), (9, 5), (11, 4)],
    'decoder': []},
   (14510, 6): {'encoder': [(1, 6), (3, 4), (5, 0), (7, 6), (9, 7), (11, 2)],
    'decoder': []},
   (540, 7): {'encoder': [(1, 3), (3, 2), (5, 3), (7, 3), (9, 3), (11, 1)],
    'decoder': []},
   (274, 8): {'encoder': [(1, 6), (3, 5), (5, 6), (7, 5), (9, 5), (11, 4)],
    'decoder': []},
   (6904, 9): {'encoder': [(1, 3), (3, 4), (5, 1), (7, 6), (9, 3), (11, 2)],
    'decoder': []},
   (12, 10

In [ ]:
converted_token_routing = {}

for batch_num in token_routing:
  converted_token_routing[batch_num] = {}
  for batch_idx in token_routing[batch_num]:
    converted_token_routing[batch_num][batch_idx] = {}
    for token_id, pos in token_routing[batch_num][batch_idx]:
      token_name = tokenizer.convert_ids_to_tokens(token_id)
      converted_token_routing[batch_num][batch_idx][(token_name, pos)] = token_routing[batch_num][batch_idx][(token_id, pos)]['encoder']


converted_token_routing

{0: {0: {('▁Sunset', 0): [(1, 7), (3, 2), (5, 2), (7, 0), (9, 3), (11, 7)],
   ('▁over', 1): [(1, 6), (3, 5), (5, 0), (7, 5), (9, 5), (11, 3)],
   ('▁the', 2): [(1, 6),
    (3, 1),
    (5, 2),
    (7, 1),
    (9, 7),
    (11, 5),
    (1, 6),
    (3, 1),
    (5, 2),
    (7, 1),
    (9, 7),
    (11, 5)],
   ('▁mountains', 3): [(1, 4), (3, 3), (5, 3), (7, 4), (9, 4), (11, 1)],
   ('▁cast', 4): [(1, 4), (3, 4), (5, 1), (7, 6), (9, 5), (11, 6)],
   ('s', 5): [(1, 0),
    (3, 5),
    (5, 4),
    (7, 3),
    (9, 0),
    (11, 6),
    (1, 0),
    (3, 0),
    (5, 4),
    (7, 3),
    (9, 0),
    (11, 3)],
   ('▁', 6): [(1, 6), (3, 0), (5, 4), (7, 3), (9, 0), (11, 3)],
   ('a', 7): [(1, 1), (3, 6), (5, 5), (7, 1), (9, 2), (11, 4)],
   ('▁golden', 8): [(1, 5), (3, 2), (5, 7), (7, 0), (9, 4), (11, 0)],
   ('▁glow', 9): [(1, 4), (3, 2), (5, 3), (7, 4), (9, 2), (11, 1)],
   ('▁across', 10): [(1, 3), (3, 5), (5, 0), (7, 5), (9, 2), (11, 3)],
   ('▁the', 11): [(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11

In [ ]:
import json

def convert_keys_to_string(input_dict):
    output_dict = {}
    for key, value in input_dict.items():
        if isinstance(key, tuple):
            # Convert tuple to string
            str_key = key[0]
        else:
            str_key = key

        if isinstance(value, dict):
            # Recursively apply to nested dictionaries
            output_dict[str_key] = convert_keys_to_string(value)
        else:
            output_dict[str_key] = value
    return output_dict

# Convert the keys of your dictionary
serializable_dict = convert_keys_to_string(converted_token_routing)

# Now you can safely serialize it to JSON
with open("/content/drive/MyDrive/same_word_token_routing_16_80.json", "w") as outfile:
    json.dump(serializable_dict, outfile)

In [ ]:
import pandas as pd

# Function to process each route tuple, keeping only the second element
def process_route(route):
    return route[1] if route else None

# Function to create a dataframe for each sentence in a batch
def create_dataframe_for_sentence(sentence_data, layer_numbers, batch_num, sentence_num):
    # Extract tokens and their routing information
    tokens, routes = zip(*sentence_data.items())
    # Process each route to keep only the second element
    processed_routes = [[process_route(route) for route in token_routes] for token_routes in routes]
    # Create a dataframe from the processed routing information
    df = pd.DataFrame(processed_routes, columns=layer_numbers)
    # Add the tokens as the first column
    df.insert(0, "Token & Position", tokens)
    # Add batch and sentence number columns
    df['Batch Number'] = batch_num
    df['Sentence Number'] = sentence_num
    return df

# Define the layer numbers as per your example
layer_numbers = [1, 3, 5, 7, 9, 11]

# Initialize an empty DataFrame to concatenate all tables
all_tables = pd.DataFrame()

# Create tables for each sentence in each batch and concatenate
for batch_num, sentences in converted_token_routing.items():
    for sentence_num, sentence_data in sentences.items():
        df = create_dataframe_for_sentence(sentence_data, layer_numbers, batch_num, sentence_num)
        all_tables = pd.concat([all_tables, df], ignore_index=True)

# Save the concatenated DataFrame to a CSV file
all_tables.to_csv("/content/drive/MyDrive/same_word_token_routing_16_80.csv", index=False)
print("Saved all tables to same_word_token_routing_16_80.csv")


Saved all tables to same_word_token_routing_16_80.csv


#PoS Tagging

In [ ]:
scrambled_sentences[0]

[['Sunset over the mountains casts a golden glow across the valley, enchanting all who see it.',
  'over casts across mountains all the a golden valley, who it. the see Sunset glow enchanting',
  'across see it. who golden the valley, a casts all the glow over mountains Sunset enchanting',
  'who the all Sunset across casts glow valley, enchanting a see mountains over the golden it.']]

In [ ]:
# original_sentences = dataset['train'][0:400]['sentence']
sentences = []
for group in scrambled_sentences:
  sentences += group[0]
original_sentences = sentences
# original_sentences = list(homonym_sentences.values())
original_sentences

['Sunset over the mountains casts a golden glow across the valley, enchanting all who see it.',
 'over casts across mountains all the a golden valley, who it. the see Sunset glow enchanting',
 'across see it. who golden the valley, a casts all the glow over mountains Sunset enchanting',
 'who the all Sunset across casts glow valley, enchanting a see mountains over the golden it.',
 "Amidst the city's heart, a quiet park provides a serene escape from its perpetual hustle and bustle.",
 "serene city's a the quiet bustle. provides its from Amidst a park perpetual escape and hustle heart,",
 "the a provides bustle. serene a quiet heart, its escape perpetual Amidst park city's and hustle from",
 "city's the a bustle. serene from Amidst quiet escape perpetual heart, hustle and a provides its park",
 'Towering ancient trees, with leaves whispering ancient tales, stand as guardians of history.',
 'leaves Towering as with ancient history. ancient stand trees, of whispering tales, guardians',
 '

In [ ]:
sentences_pos_tags = {}
for i, sentence in enumerate(original_sentences):
    tokenized_sentence = nlp(sentence)
    pos_tags = [(token.text, token.tag_) for token in tokenized_sentence]
    sentences_pos_tags[i] = pos_tags

In [ ]:
print(sentences_pos_tags[0], sentences_pos_tags[1])

[('Sunset', 'NN'), ('over', 'IN'), ('the', 'DT'), ('mountains', 'NNS'), ('casts', 'VBZ'), ('a', 'DT'), ('golden', 'JJ'), ('glow', 'NN'), ('across', 'IN'), ('the', 'DT'), ('valley', 'NN'), (',', ','), ('enchanting', 'VBG'), ('all', 'DT'), ('who', 'WP'), ('see', 'VBP'), ('it', 'PRP'), ('.', '.')] [('over', 'IN'), ('casts', 'NNS'), ('across', 'IN'), ('mountains', 'NNS'), ('all', 'PDT'), ('the', 'DT'), ('a', 'DT'), ('golden', 'JJ'), ('valley', 'NN'), (',', ','), ('who', 'WP'), ('it', 'PRP'), ('.', '.'), ('the', 'DT'), ('see', 'NN'), ('Sunset', 'NNP'), ('glow', 'VB'), ('enchanting', 'VBG')]


In [ ]:
def map_pos_tags_to_subtokens(sentences_pos_tags, converted_token_routing):
    pos_tagged_subtokens = {}

    for sentence_id, pos_tags in sentences_pos_tags.items():
        token_index = 0
        current_word = ""
        mapped_subtokens = []

        # batch_num = sentence_id // 4
        # batch_idx = sentence_id % 4

        batch_num = sentence_id
        batch_idx = 0


        for token, _ in converted_token_routing[batch_num][batch_idx].keys():
            if token.startswith('▁'):  # New word in SentencePiece
                if current_word:
                    token_index += 1
                current_word = token.lstrip('▁')
                # Assign the POS tag to the first subtoken of the new word
                mapped_subtokens.append((token, pos_tags[token_index][1]))
            else:
                # Append subtokens of the same word with the same POS tag
                current_word += token
                mapped_subtokens.append((token, pos_tags[token_index][1]))

        pos_tagged_subtokens[sentence_id] = mapped_subtokens

    return pos_tagged_subtokens

# Example usage
mapped_subtokens = map_pos_tags_to_subtokens(sentences_pos_tags, converted_token_routing)

# Output the mappings
for sentence_id, subtokens in mapped_subtokens.items():
    print(f"Sentence {sentence_id} POS-Tagged Subtokens: {subtokens}")


IndexError: list index out of range

In [ ]:
all_info = {}

for batch_num in converted_token_routing.keys():
    # for batch_idx in range(4):
    # sentence_id = batch_num * 4 + batch_idx
    sentence_id = batch_num
    batch_idx = 0
    mapped_subtokens[sentence_id]
    all_info[sentence_id] = {}

    for idx, item in enumerate(converted_token_routing[batch_num][batch_idx].items()):
        token, routing_info = item
        token_name, token_idx = token[0],token[1]
        pos_tag = mapped_subtokens[sentence_id][idx][1]
        token_key = (token_name, token_idx, pos_tag)
        all_info[sentence_id][token_key] = routing_info

IndexError: list index out of range

In [ ]:
all_info

{0: {('▁I', 0, 'PRP'): [(1, 7), (3, 1), (5, 2), (7, 6), (9, 5), (11, 5)],
  ('▁went', 1, 'VBD'): [(1, 4), (3, 4), (5, 1), (7, 6), (9, 1), (11, 6)],
  ('▁to', 2, 'IN'): [(1, 0), (3, 5), (5, 0), (7, 5), (9, 7), (11, 3)],
  ('▁the', 3, 'DT'): [(1, 3), (3, 1), (5, 2), (7, 3), (9, 7), (11, 5)],
  ('▁bank', 4, 'NN'): [(1, 4), (3, 3), (5, 3), (7, 4), (9, 3), (11, 1)],
  ('▁to', 5, 'TO'): [(1, 0), (3, 7), (5, 0), (7, 5), (9, 5), (11, 4)],
  ('▁withdraw', 6, 'VB'): [(1, 6), (3, 4), (5, 0), (7, 6), (9, 7), (11, 2)],
  ('▁money', 7, 'NN'): [(1, 3), (3, 2), (5, 3), (7, 3), (9, 3), (11, 1)],
  ('▁before', 8, 'IN'): [(1, 6), (3, 5), (5, 6), (7, 5), (9, 5), (11, 4)],
  ('▁heading', 9, 'VBG'): [(1, 3), (3, 4), (5, 1), (7, 6), (9, 3), (11, 2)],
  ('▁to', 10, 'IN'): [(1, 0), (3, 5), (5, 0), (7, 5), (9, 7), (11, 3)],
  ('▁the', 11, 'DT'): [(1, 3), (3, 1), (5, 2), (7, 1), (9, 7), (11, 0)],
  ('▁river', 12, 'NN'): [(1, 4), (3, 3), (5, 7), (7, 3), (9, 4), (11, 1)],
  ('▁bank', 13, 'NN'): [(1, 4), (3, 3), (5

In [ ]:
import json

# Convert tuples in keys to a JSON-serializable format
serializable_data = {
    k: {str(key): value for key, value in v.items()} for k, v in all_info.items()
}


path_name = "/content/drive/MyDrive/homonym_sentences_all_info_base8_50_pos_tagged."
with open(path_name, "w") as outfile:
    json.dump(serializable_data, outfile, indent=4)

# all_info.to_csv("/content/drive/MyDrive/syntax_gym_all_sentences_base8_400_pos_tagged.csv", index=False)

In [ ]:
from collections import defaultdict
import numpy as np

def group_by_pos(all_info, group_size=4):
    pos_group = defaultdict(lambda: defaultdict(list))
    for sentence_idx, tokens in all_info.items():
        sentence_group_index = sentence_idx // group_size
        for (token, token_idx, pos_tag), route in tokens.items():
            pos_group[pos_tag][sentence_group_index].append((token_idx, route))
    return pos_group

def compare_pos_instances(all_info, group_size=4):
    instance_comparisons = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    for sentence_idx, sentence_data in all_info.items():
        sentence_group_index = sentence_idx // group_size
        tag_count = defaultdict(int)

        for (token, token_idx, pos_tag), route in sentence_data.items():
            tag_count[pos_tag] += 1
            instance = tag_count[pos_tag]
            instance_comparisons[pos_tag][sentence_group_index][instance].append(route)
    return instance_comparisons

def group_and_compare_words(all_info, group_size=4):
    word_group = defaultdict(lambda: defaultdict(list))
    for sentence_idx, tokens in all_info.items():
        sentence_group_index = sentence_idx // group_size
        for (token, token_idx, pos_tag), route in tokens.items():
            word_group[token][sentence_group_index].append((token_idx, route))
    return word_group


# Example Usage:
pos_group = group_by_pos(all_info)
instance_comparisons = compare_pos_instances(all_info)
word_group = group_and_compare_words(all_info)

In [ ]:
def calculate_overlap_coefficient(grouped_data, group_size=4):
    overlap_results = defaultdict(lambda: defaultdict(float))
    for key, group_data in grouped_data.items():
        print(key)
        for group_idx in range(group_size):
            if group_idx in group_data:
                route_sets = [set(map(tuple, data[1])) for data in group_data[group_idx]]
                if route_sets:
                    intersection = set.intersection(*route_sets)
                    union = set.union(*route_sets)
                    overlap = len(intersection) / len(union) if union else 1.0
                    overlap_results[key][group_idx] = overlap
    return overlap_results

calculate_overlap_coefficient(pos_group)

In [ ]:
def calculate_overlap_coefficient(routes):
    # Convert each integer in the routes to a set containing that integer
    converted_routes = [[{route} for route in routes_list] for routes_list in routes]

    # Transpose the converted route sets to compare the same indices across all routes
    transposed_routes = list(zip(*converted_routes))

    # Calculate the overlap coefficient for each index
    overlap_coefficients = []
    print(transposed_routes)
    for index_routes in transposed_routes:
        intersection = set.intersection(*index_routes)
        smallest_set_size = min(len(index_route) for index_route in index_routes)
        if smallest_set_size > 0:
            overlap = len(intersection) / smallest_set_size
        else:
            overlap = 0
        overlap_coefficients.append(overlap)

    return overlap_coefficients


In [ ]:
instance_comparisons['DT'][0][3]

[[(1, 9), (3, 11), (5, 3), (7, 8), (9, 6), (11, 14)],
 [(1, 9), (3, 11), (5, 3), (7, 8), (9, 6), (11, 14)],
 [(1, 7), (3, 11), (5, 3), (7, 8), (9, 9), (11, 14)],
 [(1, 7), (3, 11), (5, 3), (7, 8), (9, 6), (11, 14)]]

In [ ]:
ex_maps = instance_comparisons['DT'][0][3]

In [ ]:
ex_maps

[[(1, 9), (3, 11), (5, 3), (7, 8), (9, 6), (11, 14)],
 [(1, 9), (3, 11), (5, 3), (7, 8), (9, 6), (11, 14)],
 [(1, 7), (3, 11), (5, 3), (7, 8), (9, 9), (11, 14)],
 [(1, 7), (3, 11), (5, 3), (7, 8), (9, 6), (11, 14)]]

In [ ]:
route_sets = [[expert for layer, expert in route] for route in ex_maps]

In [ ]:
route_sets

[[9, 11, 3, 8, 6, 14],
 [9, 11, 3, 8, 6, 14],
 [7, 11, 3, 8, 9, 14],
 [7, 11, 3, 8, 6, 14]]

In [ ]:
route_sets = [[8, 11, 3, 8, 6, 14],
 [8, 11, 3, 8, 6, 14],
 [7, 11, 3, 8, 9, 14],
 [7, 11, 3, 8, 6, 14]]

In [ ]:
def calculate_coefficient_of_variation_across_routes(routes):
    # Convert the list of routes to a NumPy array for easier manipulation
    routes_array = np.array(routes)

    # Calculate mean and standard deviation at each index
    means = np.mean(routes_array, axis=0)
    std_devs = np.std(routes_array, axis=0)

    # Calculate coefficient of variation (std_dev / mean) for each index
    # Note: Handle division by zero if mean is zero
    coefficients_of_variation = np.divide(std_devs, means, out=np.zeros_like(std_devs), where=means!=0)

    return coefficients_of_variation


In [ ]:
calculate_coefficient_of_variation_across_routes(route_sets)

array([0.        , 0.        , 0.        , 0.15745916, 0.        ,
       0.        ])

In [ ]:
calculate_coefficient_of_variation_across_routes(route_sets)

array([0.06666667, 0.        , 0.        , 0.        , 0.19245009,
       0.        ])

In [ ]:
calculate_coefficient_of_variation_across_routes(route_sets)

array([0.125     , 0.        , 0.        , 0.        , 0.19245009,
       0.        ])

In [ ]:
instance_comparisons['DT'][0]

defaultdict(list,
            {1: [[(1, 7), (3, 1), (5, 2), (7, 7), (9, 3), (11, 7)],
              [(1, 7), (3, 1), (5, 2), (7, 1), (9, 3), (11, 7)],
              [(1, 7), (3, 1), (5, 2), (7, 1), (9, 3), (11, 7)],
              [(1, 7), (3, 1), (5, 2), (7, 7), (9, 3), (11, 7)]],
             2: [[(1, 6), (3, 1), (5, 7), (7, 1), (9, 7), (11, 5)],
              [(1, 6), (3, 1), (5, 7), (7, 1), (9, 7), (11, 0)],
              [(1, 3), (3, 1), (5, 7), (7, 1), (9, 7), (11, 0)],
              [(1, 3), (3, 1), (5, 7), (7, 1), (9, 7), (11, 5)]],
             3: [[(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)]],
             4: [[(1, 0), (3, 0), (5, 2), (7, 1), (9, 5), (11, 5)],
              [(1, 0), (3, 0), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 0), (3, 0), (5, 2), (7, 1), (9, 7), (1

In [ ]:
from collections import Counter
def dispersion_metric(routes):
    most_common_freq = Counter(routes).most_common(1)[0][1]
    return most_common_freq / len(routes)

# Example usage with lists
routes1 = [8, 8, 7, 6]
routes2 = [9, 9, 7, 7]

dispersion1 = dispersion_metric(routes1)
dispersion2 = dispersion_metric(routes2)

print(f"Dispersion for [8, 8, 7, 7]: {dispersion1}")
print(f"Dispersion for [9, 9, 7, 7]: {dispersion2}")


Dispersion for [8, 8, 7, 7]: 0.5
Dispersion for [9, 9, 7, 7]: 0.5


In [ ]:
# Example usage
routes = instance_comparisons['DT'][0][2]

# Prepare the routes for calculation by extracting the second element of each tuple
prepared_routes = [[expert for _, expert in route] for route in routes]

# Calculate the variance
variances = calculate_variance_across_routes(prepared_routes)
print("Variances at each index:", variances)

Variances at each index: [2.25 0.   0.   0.   0.   6.25]


In [ ]:
instance_comparisons['DT'][3]

defaultdict(list,
            {0: [[(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)]],
             1: [[(1, 3), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 3), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 3), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 3), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)]],
             4: [[(1, 1), (3, 6), (5, 5), (7, 1), (9, 2), (11, 4)],
              [(1, 1), (3, 6), (5, 5), (7, 1), (9, 2), (11, 4)],
              [(1, 1), (3, 6), (5, 5), (7, 1), (9, 2), (11, 4)],
              [(1, 1), (3, 6), (5, 5), (7, 1), (9, 2), (11, 4)]],
             6: [[(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 0), (3, 1), (5, 2), (7, 1), (9, 7), (11, 5)],
              [(1, 1), (3, 1), (5, 2), (7, 1), (9, 7), (1

In [ ]:
from collections import defaultdict
import numpy as np

def calculate_vector_differences(pos_tagged_subtokens, converted_token_routing):
    # Initialize data structures
    tag_positions = defaultdict(lambda: defaultdict(list))
    vector_differences = defaultdict(lambda: defaultdict(list))
    token_comparisons = defaultdict(lambda: defaultdict(list))
    original_routes = defaultdict(lambda: defaultdict(list))
    missing_tokens_info = defaultdict(lambda: defaultdict(list))

    # Step 1: Collect routing information for each POS tag occurrence
    for sentence_id, subtokens in enumerate(pos_tagged_subtokens):
        tag_count = defaultdict(int)
        batch_num = sentence_id // 4
        batch_idx = sentence_id % 4

        for token_idx, (token, pos_tag) in enumerate(subtokens):
            tag_count[pos_tag] += 1
            position = tag_count[pos_tag] # e.g., 1st verb, 2nd verb, etc.
            token_key = (token, token_idx)

            if token_key in converted_token_routing[batch_num][batch_idx]:
                routing_info = converted_token_routing[batch_num][batch_idx][token_key]
                tag_positions[pos_tag][position].append((sentence_id, routing_info))
                token_comparisons[pos_tag][position].append((sentence_id, token))

    # Step 2: Compare routing information across sentences
    for pos_tag, positions in tag_positions.items():
        for position, sentence_infos in positions.items():
            routing_infos = [info[1] for info in sentence_infos]
            if len(routing_infos) > 1:
                mean_vector = np.mean(routing_infos, axis=0)
                for sentence_id, routing_info in sentence_infos:
                    difference = np.abs(mean_vector - routing_info)
                    vector_differences[pos_tag][position].append((sentence_id, difference))
                    original_routes[pos_tag][position].append((sentence_id, routing_info))
            else:
                # Record missing POS tag instances
                missing_tokens_info[pos_tag][position] = [sentence_id for sentence_id, _ in sentence_infos]

    return vector_differences, token_comparisons, original_routes, missing_tokens_info

# Example usage
vector_diffs, token_comparisons, original_routes, missing_tokens_info = calculate_vector_differences(list(mapped_subtokens.values())[:4], converted_token_routing)

# Output the vector differences and additional information
for pos_tag, positions in vector_diffs.items():
    for position, differences in positions.items():
        print(f"POS Tag: {pos_tag}, Position: {position}")
        print("Tokens Compared:")
        for sentence_id, token in token_comparisons[pos_tag][position]:
            print(f"  Sentence {sentence_id}: '{token}'")
        print("Original Routes:")
        for sentence_id, route in original_routes[pos_tag][position]:
            print(f"  Sentence {sentence_id}: {'  '.join(map(str, route))}")
        print("Vector Differences:")
        for sentence_id, difference in differences:
            print(f"  Sentence {sentence_id}: {'  '.join(map(lambda diff: '[' + ' '.join(map(str, diff)) + ']', difference))}")
        if position in missing_tokens_info[pos_tag]:
            missing_sentences = ', '.join(map(str, missing_tokens_info[pos_tag][position]))
            print(f"Missing in sentences: {missing_sentences}")
        print()
        print("----------------------------------------------")
        print()


POS Tag: DT, Position: 1
Tokens Compared:
  Sentence 0: '▁The'
  Sentence 1: '▁The'
  Sentence 2: '▁The'
  Sentence 3: '▁The'
Original Routes:
  Sentence 0: (1, 7)  (3, 1)  (5, 2)  (7, 7)  (9, 3)  (11, 7)
  Sentence 1: (1, 7)  (3, 1)  (5, 2)  (7, 1)  (9, 3)  (11, 7)
  Sentence 2: (1, 7)  (3, 1)  (5, 2)  (7, 1)  (9, 3)  (11, 7)
  Sentence 3: (1, 7)  (3, 1)  (5, 2)  (7, 7)  (9, 3)  (11, 7)
Vector Differences:
  Sentence 0: [0.0 0.0]  [0.0 0.0]  [0.0 0.0]  [0.0 3.0]  [0.0 0.0]  [0.0 0.0]
  Sentence 1: [0.0 0.0]  [0.0 0.0]  [0.0 0.0]  [0.0 3.0]  [0.0 0.0]  [0.0 0.0]
  Sentence 2: [0.0 0.0]  [0.0 0.0]  [0.0 0.0]  [0.0 3.0]  [0.0 0.0]  [0.0 0.0]
  Sentence 3: [0.0 0.0]  [0.0 0.0]  [0.0 0.0]  [0.0 3.0]  [0.0 0.0]  [0.0 0.0]

----------------------------------------------

POS Tag: DT, Position: 2
Tokens Compared:
  Sentence 0: '▁the'
  Sentence 1: '▁the'
  Sentence 2: '▁the'
  Sentence 3: '▁the'
Original Routes:
  Sentence 0: (1, 6)  (3, 1)  (5, 7)  (7, 1)  (9, 7)  (11, 5)
  Sentence 1: (1, 6

In [ ]:
calculate_vector_differences(list(mapped_subtokens.values())[4:8], converted_token_routing)

(defaultdict(<function __main__.calculate_vector_differences.<locals>.<lambda>()>,
             {'DT': defaultdict(list,
                          {1: [array([[0., 0.],
                                   [0., 0.],
                                   [0., 0.],
                                   [0., 3.],
                                   [0., 0.],
                                   [0., 0.]]),
                            array([[0., 0.],
                                   [0., 0.],
                                   [0., 0.],
                                   [0., 3.],
                                   [0., 0.],
                                   [0., 0.]]),
                            array([[0., 0.],
                                   [0., 0.],
                                   [0., 0.],
                                   [0., 3.],
                                   [0., 0.],
                                   [0., 0.]]),
                            array([[0., 0.],
               

In [ ]:
# Example usage
vector_diffs, token_comparisons, original_routes, length_mismatches = calculate_vector_differences(list(mapped_subtokens.values())[8:12], converted_token_routing)

# Output the vector differences and additional information
for pos_tag, positions in vector_diffs.items():
    for position, differences in positions.items():
        print(f"POS Tag: {pos_tag}, Position: {position}")
        print(f"Tokens Compared: {token_comparisons[pos_tag][position]}")
        print("Original Routes:")
        for route in original_routes[pos_tag][position]:
            print(f"  {route}")
        print(f"Vector Differences: {differences}")
        if position in length_mismatches[pos_tag]:
            print(f"Length Mismatches: {length_mismatches[pos_tag][position]}")
        print()


ValueError: too many values to unpack (expected 4)

In [ ]:
list(mapped_subtokens.values())[:4]

[[('▁The', 'DT'),
  ('▁woman', 'NN'),
  ('▁brought', 'VBD'),
  ('▁the', 'DT'),
  ('▁sandwich', 'NN'),
  ('▁from', 'IN'),
  ('▁the', 'DT'),
  ('▁kitchen', 'NN'),
  ('▁with', 'IN'),
  ('▁', 'DT'),
  ('a', 'DT'),
  ('▁new', 'JJ'),
  ('▁microwave', 'NN'),
  ('▁fell', 'VBD'),
  ('▁in', 'IN'),
  ('▁the', 'DT'),
  ('▁dining', 'NN'),
  ('▁room', 'NN'),
  ('</s>', 'NN'),
  ('<pad>', 'NN'),
  ('<pad>', 'NN')],
 [('▁The', 'DT'),
  ('▁woman', 'NN'),
  ('▁who', 'WP'),
  ('▁was', 'VBD'),
  ('▁brought', 'VBN'),
  ('▁the', 'DT'),
  ('▁sandwich', 'NN'),
  ('▁from', 'IN'),
  ('▁the', 'DT'),
  ('▁kitchen', 'NN'),
  ('▁with', 'IN'),
  ('▁', 'DT'),
  ('a', 'DT'),
  ('▁new', 'JJ'),
  ('▁microwave', 'NN'),
  ('▁fell', 'VBD'),
  ('▁in', 'IN'),
  ('▁the', 'DT'),
  ('▁dining', 'NN'),
  ('▁room', 'NN'),
  ('</s>', 'NN')],
 [('▁The', 'DT'),
  ('▁woman', 'NN'),
  ('▁given', 'VBD'),
  ('▁the', 'DT'),
  ('▁sandwich', 'NN'),
  ('▁from', 'IN'),
  ('▁the', 'DT'),
  ('▁kitchen', 'NN'),
  ('▁with', 'IN'),
  ('▁', 'DT'),


In [ ]:
import numpy as np

def calculate_vector_differences(sentences_pos_tags, converted_token_routing):
    # Step 1: Group Tokens by POS Tags
    pos_grouped_tokens = {}
    for sentence_id, tags in sentences_pos_tags.items():
        batch_num = sentence_id // 4
        batch_idx = sentence_id % 4

        for idx, (token, pos_tag) in enumerate(tags):
            if pos_tag not in pos_grouped_tokens:
                pos_grouped_tokens[pos_tag] = {}
            if sentence_id not in pos_grouped_tokens[pos_tag]:
                pos_grouped_tokens[pos_tag][sentence_id] = []

            token_key = (token, idx)
            if token_key in converted_token_routing[batch_num][batch_idx]:
                pos_grouped_tokens[pos_tag][sentence_id].append(token_key)

    # Step 2: Retrieve Token Routes and Calculate Vector Differences
    vector_differences = {}
    for pos_tag, sentences in pos_grouped_tokens.items():
        route_vectors = []
        for sentence_id, tokens in sentences.items():
            for token_key in tokens:
                batch_num = sentence_id // 4
                batch_idx = sentence_id % 4
                route_vector = np.array(converted_token_routing[batch_num][batch_idx][token_key])
                route_vectors.append(route_vector)

        if route_vectors:
            # Calculating mean route vector for each POS tag
            mean_vector = np.mean(route_vectors, axis=0)
            # Calculate variance or standard deviation as a measure of difference
            vector_variance = np.var(route_vectors, axis=0)
            vector_differences[pos_tag] = vector_variance

    return vector_differences

# Example usage
vector_diffs = calculate_vector_differences(mapped_subtokens, converted_token_routing)

# Output the vector differences
for pos_tag, diff in vector_diffs.items():
    print(f"POS Tag: {pos_tag} -> Vector Difference: {diff}")


POS Tag: DT -> Vector Difference: [[0.         8.26835806]
 [0.         3.75438211]
 [0.         3.26421152]
 [0.         2.28030818]
 [0.         5.11310783]
 [0.         4.79864844]]
POS Tag: NN -> Vector Difference: [[0.         2.27252165]
 [0.         3.15111962]
 [0.         2.62101634]
 [0.         3.0972341 ]
 [0.         2.85439869]
 [0.         5.52042699]]
POS Tag: VBD -> Vector Difference: [[0.         2.34955942]
 [0.         1.07706559]
 [0.         1.3603996 ]
 [0.         0.98538421]
 [0.         5.87293119]
 [0.         3.51021581]]
POS Tag: IN -> Vector Difference: [[0.         2.41035636]
 [0.         3.050204  ]
 [0.         4.39543215]
 [0.         1.67932997]
 [0.         2.48228214]
 [0.         3.2539306 ]]
POS Tag: JJ -> Vector Difference: [[0.         1.43330052]
 [0.         2.64591012]
 [0.         4.65795197]
 [0.         2.71843593]
 [0.         4.04248987]
 [0.         5.79299924]]
POS Tag: WP -> Vector Difference: [[ 0.          0.56      ]
 [ 0.        

In [ ]:
import pandas as pd
# Function to process each route tuple, keeping only the second element
def process_route(route):
    return route[1] if route else None

# Function to create a dataframe for each sentence in a batch
def create_dataframe_for_sentence(sentence_data, layer_numbers):
    # Extract tokens and their routing information
    tokens, routes = zip(*sentence_data.items())
    # Process each route to keep only the second element
    processed_routes = [[process_route(route) for route in token_routes] for token_routes in routes]
    # Create a dataframe from the processed routing information
    df = pd.DataFrame(processed_routes, columns=layer_numbers)
    # Add the tokens as the first column
    df.insert(0, "Token & Position", tokens)
    return df

# Define the layer numbers as per your example
layer_numbers = [1, 3, 5, 7, 9, 11]

# Create and display tables for each sentence in each batch
for batch_num, sentences in converted_token_routing.items():
    print(f"Batch {batch_num} Tables:")
    for sentence_num, sentence_data in sentences.items():
        df = create_dataframe_for_sentence(sentence_data, layer_numbers)
        print(f"Sentence {sentence_num} Table:")
        print(df)
        print("\n")


Batch 0 Tables:
Sentence 0 Table:
    Token & Position  1  3  5  7  9  11
0          (▁The, 0)  7  1  2  7  3   7
1        (▁woman, 1)  4  2  7  2  3   1
2      (▁brought, 2)  4  4  1  6  7   6
3          (▁the, 3)  6  1  7  1  7   5
4     (▁sandwich, 4)  5  2  3  3  4   1
5         (▁from, 5)  3  5  0  5  5   3
6          (▁the, 6)  0  1  2  1  7   5
7      (▁kitchen, 7)  4  3  3  4  3   7
8         (▁with, 8)  3  5  0  5  5   4
9             (▁, 9)  0  0  2  1  5   5
10           (a, 10)  1  6  5  1  2   4
11        (▁new, 11)  5  2  7  2  3   0
12  (▁microwave, 12)  5  2  3  0  3   1
13       (▁fell, 13)  4  4  1  6  5   7
14         (▁in, 14)  3  5  0  5  5   3
15        (▁the, 15)  0  1  2  1  7   5
16     (▁dining, 16)  5  2  3  4  4   7
17       (▁room, 17)  4  3  3  3  6   0
18        (</s>, 18)  7  7  5  7  3   4
19       (<pad>, 19)  4  1  3  4  2   4
20       (<pad>, 20)  4  1  3  4  2   2


Sentence 1 Table:
    Token & Position  1  3  5  7  9  11
0          (▁The, 0)  7  1

In [ ]:
import plotly.express as px
import pandas as pd

# Convert token_routing data to a DataFrame
data = []
for batch_id, batch_data in converted_token_routing.items():
    for token, paths in batch_data.items():
        for path in paths['encoder']:
            data.append({'batch_id': batch_id, 'token': token, 'layer': path[0], 'expert': path[1]})

df = pd.DataFrame(data)

# Create a line plot
fig = px.line(df, x='layer', y='expert', color='token', line_group='batch_id', hover_name='token')

# Customize the layout
fig.update_layout(title='Token Routing Trajectories', xaxis_title='Layer', yaxis_title='Expert Index')

# Show the plot
fig.show()


# **BASE-16**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/switch-base-16")
model = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-16")

pytorch_model.bin:   0%|          | 0.00/4.29G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def router_forward_hook(layer_id, layer_type, token_routing):
    def hook(module, input, output):
        global original_input_ids, batch_number
        expert_index = output[1][1]

        batch_size, seq_length = expert_index.shape
        for batch_idx in range(batch_size):
            for token_idx in range(seq_length):
                token_id = original_input_ids[batch_idx, token_idx].item()
                token_key = (token_id, token_idx)  # Unique identifier for each token occurrence

                if batch_number not in token_routing:
                  token_routing[batch_number] = {}
                if batch_idx not in token_routing[batch_number]:
                    token_routing[batch_number][batch_idx] = {}
                if token_key not in token_routing[batch_number][batch_idx]:
                    token_routing[batch_number][batch_idx][token_key] = {"encoder": [], "decoder": []}

                route = (layer_id, expert_index[batch_idx, token_idx].item())
                token_routing[batch_number][batch_idx][token_key][layer_type].append(route)

    return hook


In [ ]:
hooks = []
# Attach the router forward hooks to the encoder layers
token_routing_16 = {}
for i, block in enumerate(model.encoder.block):
      if i % 2 == 1:
        hook_handle = block.layer[1].mlp.register_forward_hook(router_forward_hook(i, "encoder", token_routing_16))
        hooks.append(hook_handle)

# for i, block in enumerate(model.encoder.block):
#       if i % 2 == 1:
#         hook = router_forward_hook(i, "encoder", input_ids, token_routing)
#         block.layer[1].mlp.router.register_forward_hook(hook)

In [ ]:
original_input_ids = None
batch_number = 0

# Function to process a batch of sentences
def process_batch(batch_sentences):
    # Tokenize the input sentences
    global original_input_ids
    inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True)

    # Run the model
    with torch.no_grad():
        original_input_ids = inputs['input_ids']
        model.generate(original_input_ids, max_length=20)

# Process the sentences in batches of 4
batch_size = 4
for i in range(0, len(sentences), batch_size):
    batch_number = i // 4
    batch_sentences = sentences[i:i + batch_size]
    process_batch(batch_sentences)

# # Print or analyze the routing decisions
# print(token_routing)

In [ ]:
for hook in hooks:
    hook.remove()


In [ ]:
token_routing_16

In [ ]:
converted_token_routing_16 = {}

for batch_num in token_routing_16:
  converted_token_routing_16[batch_num] = {}
  for batch_idx in token_routing_16[batch_num]:
    converted_token_routing_16[batch_num][batch_idx] = {}
    for token_id, pos in token_routing_16[batch_num][batch_idx]:
      token_name = tokenizer.convert_ids_to_tokens(token_id)
      converted_token_routing_16[batch_num][batch_idx][(token_name, pos)] = token_routing_16[batch_num][batch_idx][(token_id, pos)]['encoder']


converted_token_routing_16

In [ ]:
import json

def convert_keys_to_string(input_dict):
    output_dict = {}
    for key, value in input_dict.items():
        if isinstance(key, tuple):
            # Convert tuple to string
            str_key = key[0]
        else:
            str_key = key

        if isinstance(value, dict):
            # Recursively apply to nested dictionaries
            output_dict[str_key] = convert_keys_to_string(value)
        else:
            output_dict[str_key] = value
    return output_dict

# Convert the keys of your dictionary
serializable_dict = convert_keys_to_string(converted_token_routing_16)

# Now you can safely serialize it to JSON
with open("/content/drive/MyDrive/syntax_gym_token_routing_16_200.json", "w") as outfile:
    json.dump(serializable_dict, outfile)

In [ ]:
import pandas as pd

# Function to process each route tuple, keeping only the second element
def process_route(route):
    return route[1] if route else None

# Function to create a dataframe for each sentence in a batch
def create_dataframe_for_sentence(sentence_data, layer_numbers, batch_num, sentence_num):
    # Extract tokens and their routing information
    tokens, routes = zip(*sentence_data.items())
    # Process each route to keep only the second element
    processed_routes = [[process_route(route) for route in token_routes] for token_routes in routes]
    # Create a dataframe from the processed routing information
    df = pd.DataFrame(processed_routes, columns=layer_numbers)
    # Add the tokens as the first column
    df.insert(0, "Token & Position", tokens)
    # Add batch and sentence number columns
    df['Batch Number'] = batch_num
    df['Sentence Number'] = sentence_num
    return df

# Define the layer numbers as per your example
layer_numbers = [1, 3, 5, 7, 9, 11]

# Initialize an empty DataFrame to concatenate all tables
all_tables = pd.DataFrame()

# Create tables for each sentence in each batch and concatenate
for batch_num, sentences in converted_token_routing_16.items():
    for sentence_num, sentence_data in sentences.items():
        df = create_dataframe_for_sentence(sentence_data, layer_numbers, batch_num, sentence_num)
        all_tables = pd.concat([all_tables, df], ignore_index=True)

# Save the concatenated DataFrame to a CSV file
all_tables.to_csv("/content/drive/MyDrive/syntax_gym_all_sentences_16_200.csv", index=False)
print("Saved all tables to all_sentences_16_200_tables.csv")


Saved all tables to all_sentences_16_200_tables.csv


In [ ]:
import pandas as pd
# Function to process each route tuple, keeping only the second element
def process_route(route):
    return route[1] if route else None

# Function to create a dataframe for each sentence in a batch
def create_dataframe_for_sentence(sentence_data, layer_numbers):
    # Extract tokens and their routing information
    tokens, routes = zip(*sentence_data.items())
    # Process each route to keep only the second element
    processed_routes = [[process_route(route) for route in token_routes] for token_routes in routes]
    # Create a dataframe from the processed routing information
    df = pd.DataFrame(processed_routes, columns=layer_numbers)
    # Add the tokens as the first column
    df.insert(0, "Token & Position", tokens)
    return df

# Define the layer numbers as per your example
layer_numbers = [1, 3, 5, 7, 9, 11]

# Create and display tables for each sentence in each batch
for batch_num, sentences in converted_token_routing_16.items():
    print(f"Batch {batch_num} Tables:")
    for sentence_num, sentence_data in sentences.items():
        df = create_dataframe_for_sentence(sentence_data, layer_numbers)
        print(f"Sentence {sentence_num} Table:")
        print(df)
        print("\n")


Batch 0 Tables:
Sentence 0 Table:
    Token & Position   1   3   5   7   9  11
0          (▁The, 0)   5   3   3   7   8   3
1        (▁woman, 1)   4   1  12  11   4  10
2      (▁brought, 2)  13  13   4   4   9   6
3          (▁the, 3)   7   3   3  15   9   3
4     (▁sandwich, 4)   6   4   1  15  13   2
5         (▁from, 5)   0   7   8  14   1   7
6          (▁the, 6)   9  11   3   8   6  14
7      (▁kitchen, 7)   4   4   1   9   7   0
8         (▁with, 8)  11   7   9  12   3   8
9             (▁, 9)   8   3   4  10   3   3
10           (a, 10)   3   5  14   6   2   0
11        (▁new, 11)  11   6   6   3   2   5
12  (▁microwave, 12)   6   1   1   9   7  11
13       (▁fell, 13)  13  13   4  13  14   6
14         (▁in, 14)   0  15   8   3   1   7
15        (▁the, 15)   7  11   3   2   9   3
16     (▁dining, 16)   6  11   1   9  13  10
17       (▁room, 17)   4  15   1   1   5  11
18        (</s>, 18)   5   8  15   2   7  11
19       (<pad>, 19)  12   0  15   3   3   8
20       (<pad>, 20) 

#**SCRATCH**

In [ ]:
# def router_forward_hook(layer_id, layer_type, original_input_ids, token_routing):
#     def hook(module, input, output):
#         _, _, router_logits = output
#         expert_index = router_logits.argmax(-1)

#         # Handle the sequence length based on the layer type
#         token_indices = range(original_input_ids.size(1)) if layer_type == "encoder" else [-1]

#         for i in token_indices:
#             # Use original token IDs for tracking
#             token_id = original_input_ids[0, i].item()
#             if token_id not in token_routing:
#                 token_routing[token_id] = {"encoder": [], "decoder": []}

#             # Record the routing decision
#             route = (layer_id, expert_index[0, 0 if layer_type == "decoder" else i].item())
#             token_routing[token_id][layer_type].append(route)

#     return hook

# def lm_head_hook(routing_decisions, token_ids):
#     def hook(module, input, output):
#         # The output here are the logits, from which the next token is determined
#         next_token_logits = output
#         next_token_id = next_token_logits.argmax(-1).item()

#         # Update the stored routing decisions with the correct token ID
#         for decision in routing_decisions:
#             if decision[2] is None:
#                 decision[2] = next_token_id

#         # Store this token ID for analysis later
#         if next_token_id not in token_ids:
#             token_ids[next_token_id] = []
#         token_ids[next_token_id].extend(routing_decisions)

#         # Clear the routing decisions for the next token generation
#         routing_decisions.clear()

#     return hook



In [ ]:
# # Lists to store routing decisions and token IDs
# routing_decisions = []
# token_ids = {}

# # Attach feed-forward hooks to each feed-forward layer in the decoder
# for i, block in enumerate(model.decoder.block):
#     if hasattr(block, 'SwitchTransformersSparseMLP'):
#         hook = feed_forward_hook(i, routing_decisions)
#         block.SwitchTransformersSparseMLP.router.register_forward_hook(hook)

# # Attach the lm_head hook
# model.lm_head.register_forward_hook(lm_head_hook(routing_decisions, token_ids))


In [ ]:
# # Initialize a dictionary to store routing information using token IDs
# token_routing = {}

# # Attach hooks to the encoder and decoder layers
# for i, block in enumerate(model.encoder.block):
#       if i % 2 == 1:
#         hook = router_forward_hook(i, "encoder", input_ids, token_routing)
#         block.layer[1].mlp.router.register_forward_hook(hook)

# for i, block in enumerate(model.decoder.block):
#       if i % 2 == 1:
#         hook = router_forward_hook(i, "decoder", input_ids, token_routing)
#         block.layer[2].mlp.router.register_forward_hook(hook)

# # Attach the lm_head hook
# model.lm_head.register_forward_hook(lm_head_hook(routing_decisions, token_ids))


In [ ]:
# outputs = model.generate(input_ids)

# # Convert token IDs to token names and reorganize the routing data
# final_routing = {}
# for token_id, routing_info in token_routing.items():
#     token_name = tokenizer.decode([token_id])
#     final_routing[token_name] = routing_info

# # Now final_routing contains the routing information with token names


In [ ]:
tokenizer.decode(outputs[0])

'<pad><extra_id_0>.<extra_id_1> a new era of a new era of a new '

In [ ]:
# Run the model for inference
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
token_routes['Mix']

{'encoder': [(1, 7), (3, 2), (5, 7), (7, 0), (9, 4), (11, 7)], 'decoder': []}

In [ ]:
token_routes['ture']

{'encoder': [(1, 0), (3, 6), (5, 4), (7, 3), (9, 6), (11, 1)], 'decoder': []}

In [ ]:
len(token_routes)

16

In [ ]:
len(token_routes['This']['decoder'])

114

In [ ]:
token_routes

In [ ]:
input_ids

tensor([[  100,    19,     3,     9,   794,    12,   734,   149,  7382,  2693,
            18,   858,    18, 29906, 19790,     5,     1]])

In [ ]:
# def router_forward_hook(layer_id, layer_type):
#     def hook(module, input, output):
#         # Get the expert index for each token
#         _, _, router_logits = output
#         expert_index = torch.argmax(router_logits, dim=-1)

#         # Update the token_routes dictionary
#         for token_id, expert_id in enumerate(expert_index.tolist()):
#             if token_id not in token_routes:
#                 token_routes[token_id] = []
#             token_routes[token_id].append((layer_type, layer_id, expert_id))
#     return hook

# # Initialize the token_routes dictionary
# token_routes = {}

# # Register the hooks for each router in the encoder and decoder layers
# for i, block in enumerate(model.encoder.block):
#     if i % 2 == 1:
#       block.layer[1].mlp.router.register_forward_hook(router_forward_hook(i, "encoder"))


# for i, block in enumerate(model.decoder.block):
#       if i % 2 == 1:
#         block.layer[2].mlp.router.register_forward_hook(router_forward_hook(i, "decoder"))

In [ ]:
# sentence1 =

input_text = "How is it going world, we are out here celebrating!"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

In [ ]:
# def router_forward_hook(layer_id, layer_type, original_input_ids):
#     def hook(module, input, output):
#         _, _, router_logits = output
#         expert_index = router_logits.argmax(-1)
#         print(expert_index)

#         # Ensure that the number of tokens matches between expert_index and original_input_ids
#         num_tokens = expert_index.size(1)  # Assuming batch_size x seq_len shape

#         for i in range(num_tokens):
#             token_id = original_input_ids[0, i].item()  # Get the token ID for the i-th token
#             token_name = tokenizer.decode([token_id])   # Decode the token ID to get the token name

#             if token_name not in token_routes:
#                 token_routes[token_name] = {"encoder": [], "decoder": []}

#             route = (layer_id, expert_index[0, i].item())  # Get the expert index for the i-th token

#             if layer_type == "encoder":
#                 token_routes[token_name]["encoder"].append(route)
#             else:
#                 token_routes[token_name]["decoder"].append(route)

#         return None

#     return hook


# # Initialize the token_routes dictionary
# token_routes = {}

# # Register the hooks for each router in the encoder and decoder layers
# for i, block in enumerate(model.encoder.block):
#     if i % 2 == 1:
#       block.layer[1].mlp.router.register_forward_hook(router_forward_hook(i, "encoder", input_ids))


# for i, block in enumerate(model.decoder.block):
#       if i % 2 == 1:
#         block.layer[2].mlp.router.register_forward_hook(router_forward_hook(i, "decoder", input_ids))

In [ ]:
# input_text = "How is it going world, we are out here celebrating!"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids
# inputs = tokenizer(input_text, return_tensors='pt')
outputs = model.generate(input_ids)